# Basic model construction

## Introduction
The objective of this first notebook is to introduce the _summer_ interface 
that we will use throughout this series,
use it to demonstrate the construction of a simple model,
and demonstrate several principles of infectious disease modelling.
Because this is the first notebook/chapter, 
there is slightly more content in this notebook introducing the code packages we will be using.
Although the model is so simple that we could actually write this system even more tersely,
we'll also try to use good coding habits from the outset.
Also, we'll immediately come back to some of the features of our simple model in the following notebooks,
including thinking more about flows and how we obtain numerical solutions for our system.
In our example model, we will create an SIR compartmental model for a general, unspecified, immunising infectious disease spreading through a fully susceptible population.

In this model there will be:

- Three compartments, named `susceptible`, `infectious` and `recovered`
- A starting population of 1000 people, with 10 of them infected (and infectious)
- An evaluation timespan from day zero to 20 units of time
- Inter-compartmental flows for infection, death and recovery

## Preliminaries
As with any Python script, we first have to import the objects we need,
plus we'll set the visualisation of pandas objects to be interactive.
You'll see similar cells to these in all the notebooks in the series.

In [ ]:
# If running on Google Colab, run the following line of code to install the summer package
# %pip install summerepi2

In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

## Model definition
First, let's create a function that gives us a base SIR model with the
basic compartments, starting populations and inter-compartmental flows implemented.
Note the steps in model construction included as comments through the following function.

In [ ]:
def get_sir_model(
    model_config: dict,
) -> CompartmentalModel:
    """
    Generate an instance of an SIR model with some standard parameters, 
    population distribution and parameters hard-coded.
    
    Args:
        model_config: Values needed for model construction other than the parameter values   
    Returns:
        The summer model object
    """
    
    # Name the base model compartments
    compartments = (
        "susceptible",
        "infectious",
        "recovered",
    )
    
    # Specify the evaluation times
    analysis_times = (
        model_config["start_time"], 
        model_config["end_time"],
    )
    
    # Build the basic models with this information and specify which compartment is infectious
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=["infectious"],
    )
    
    # Assign the starting population
    model.set_initial_population(
        distribution=
        {
            "susceptible": model_config["population"] - model_config["seed"], 
            "infectious": model_config["seed"],
        }
    )
    
    # Add a dynamic infection flow that transitions people from susceptible to infectious
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("contact_rate"),
        source="susceptible", 
        dest="infectious",
    )
    
    # Add a constant flow that transitions people from infectious to recovered
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=Parameter("recovery"),
        source="infectious", 
        dest="recovered",
    )
    
    # Add a constant death flow that transitions people from infectious out of the model
    model.add_death_flow(
        name="infection_death", 
        death_rate=Parameter("infection_death"),
        source="infectious",
    )
    
    # Return the finished model object
    return model

## Getting the model, running it and examining the results
Now we can use our model building function to get an instance of this model,
run it, and have a look at the compartment size progression over time.
Note that we use the plotting functions built-in to pandas objects to do this.
Pandas is a very widely used library for data processing, which we will use extensively in this series.
Because there may be considerable output wrangling necessary after we have run our model and this wrangling is not infectious diseases-specific,
we prefer use external libraries for these steps.
Here we use pandas' easy integration with _plotly_ to create an interactive plot from the pandas
object in a single line of code.
(Note that this could easily be set to other output formats,
e.g. by setting the pandas plotting backend to _matplotlib_ instead.)

In [ ]:
config = {
    "population": 1000.,
    "seed": 10.,
    "start_time": 0.,
    "end_time": 20.,
}

parameters = {
    "recovery": 0.333,
    "infection_death": 0.05,
    "contact_rate": 1.,
}

sir_model = get_sir_model(config)  # Get the model object
sir_model.run(parameters=parameters)  # Run it
compartment_values = sir_model.get_outputs_df()  # Access the compartment values as a pandas dataframe
compartment_values.plot()  # Plot

## Digging into the model object
Now that we have our `CompartmentalModel` object,
we can use this structure to inspect some aspects of what is going on under the surface,
for example, compartments, flows and others attributes.
This is **highly recommended**, 
to ensure that the model you have created is consistent with what you were wanting.
Try out using tab complete in this notebook to inspect the range of methods and
attributes that are available for a `CompartmentalModel` object.

In [ ]:
print(sir_model.compartments)
print(sir_model.initial_population)
print(sir_model.times)
compartment_values

## Epidemiological messages
This is clearly a very simple model of an epidemic caused by a short-lived pathogen that induces complete immunity in its host.
However, despite its simplicity, it does capture a surprising number of the actual features of an epidemic caused by an infection
of this type.
In general, models of infectious diseases transmission should be as complicated as they need to be,
which means that the additional complexity that we might need to inject into this model is highly dependent on the purpose that
we will be using it for - or the epidemiological question that we will be addressing through our analysis.
It may also be dependent on us having sufficient epidemiological understanding of the epidemic to be able to incorporate these
features - with a reasonable level of confidence that we are actually capturing the processes that we are interested in
(including empiric data to estimate the parameters that we need to build our more complicated model).

Let's think of some of the epidemiological features that this very simple model **does** capture:
- Very broadly, this model gives us the shape that epi curves often follow - looking vaguely like a bell
- There is an exponential growth phase when the population remains largely susceptible
- The growth in the epidemic decreases as the proportion of susceptibles decreases
- As the effective reproduction number falls past one, the epidemic peaks and begins to decline
- The epidemic dies out as the susceptibles decrease and the effective reproduction number drops below one
- As the epidemic ends and transmission declines to approximately zero, susceptibles are depleted, but not completely - a proportion of the population remains susceptible even after the epidemic

Let's think of some of the epidemiological features that this model **does not** capture:
- Any heterogeneity in the background population with regards progression through infection states after exposure
- Any heterogeneity in transmission, such as greater transmission to people within the population with similar characteristics
- Any heterogeneity in the pathogen, such as multiple strains with different characteristics circulating through the population
- Any changes in how people transition through their stages over time that might be induced through changes external to the model
(i.e. other than those related to the changes in the population distribution across compartments resulting from transmission of this immunising pathogen)
- Tracking any outputs other than the sizes of the model's compartments

We will return to these features and how to elaborate our base model to capture them over the following notebooks.